# Attacker model

Here we will put it all togheter using the generated synthetic data from `data_synthesis_playground.ipynb`

In [1]:
from mblearn import AttackModels, ShadowModels

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

First we need to make a target model (we will use RandomForest with 100 estimators)

## Target model

In [3]:
rf_target = RandomForestClassifier(n_estimators=100)
data, target = load_wine(return_X_y=True)



In [4]:
scaler = MinMaxScaler()
data_std = scaler.fit_transform(data)

# split to test membership in X_train
X_train, X_test, y_train, y_test = train_test_split(data_std, target, test_size=0.2)

rf_target.fit(X_train, y_train);

## Shadow model

Now train the Shadow models with synthetic data and the same learner (that's cheating a bit)

In [5]:
synth_data = pd.read_csv('synthetic_data.csv')
synth_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
0,0.648708,0.556606,0.475915,0.604599,0.296312,0.575440,0.590239,0.994838,0.260606,0.326840,0.892040,0.653399,0.449623,0
1,0.954101,0.608156,0.793287,0.375929,0.854361,0.659643,0.912113,0.117836,0.603246,0.645942,0.115053,0.860870,0.656279,0
2,0.618762,0.365778,0.232566,0.434179,0.800469,0.493136,0.386309,0.298686,0.182332,0.464039,0.449998,0.571479,0.949356,0
3,0.976243,0.462703,0.850762,0.347162,0.848853,0.698433,0.866704,0.552660,0.723015,0.372361,0.396304,0.549000,0.321202,0
4,0.765457,0.635224,0.525063,0.009490,0.581303,0.718589,0.749601,0.134644,0.388569,0.550524,0.408401,0.797959,0.753048,0


In [6]:
rf_shadow =  RandomForestClassifier(n_estimators=100)

In [74]:
sh = ShadowModels(4, synth_data, 3, rf_shadow)

In [75]:
shadow_data = sh.results

## Attacker

Now that we have the shadow dataset we can train the attacker model. The attacker learner doesn't need to be the same as the target so pick the one that performs the best.

`AttackModels` trains a model for each original class in shadow data (and in target model) with the in/out of training label as the target label.

In [76]:
rf_attack = RandomForestClassifier(n_estimators=100)

In [77]:
attacker = AttackModels(target_classes=3, attack_learner=rf_attack)

In [78]:
attacker.fit(shadow_data)

now lets test with all `X_train` and `X_test`

### Test accuracy

In [79]:
# predict with data that was in the target training set 
X_in = rf_target.predict_proba(X_train)

res_in = attacker.predict(X_in, y_train, batch=True)

In [80]:
# predict with target's test data 
X_out = rf_target.predict_proba(X_test)

res_out = attacker.predict(X_out, y_test, batch=True)

proportion of records correctly predicted as **members** of the training set

In [81]:
np.sum(np.argmax(res_in, axis=1)) / len(res_in)

0.9154929577464789

records predicted as **not members** of the training set

In [83]:
1 - np.sum(np.argmax(res_out, axis=1)) / len(res_out)

0.33333333333333337

No big deal but for a toy example it's something! Simple and straight forward.